### Understanding the PyTorch parallel scan, with RNNs/Mamba in mind

TODO's : 
- sections
- proof, in a separate file

This file gives a detailed background and explanation of the `pscan.py` file. The goal is to impement a parallel scan in PyTorch.

In [1]:
import torch
import math

First of all, what is a <b>scan</b> ?

A scan is defined as an operation that takes as input an array and procudes an array as output. You can see that it is quite general.

 A simple and well-known example of a scan is the <i>cumulative sum of an array</i> :

In [2]:
X = torch.tensor([1, 2, 3, 4])

torch.cumsum(X, dim=0)

tensor([ 1,  3,  6, 10])

For the rest of this document, we will denote `L` as the length of our input array `X`.

The most basic way to implement a scan is to use a simple for loop :

In [3]:
Y = torch.zeros_like(X)

cumulative_sum = 0
for t in range(X.size(0)):
    cumulative_sum += X[t]
    Y[t] = cumulative_sum

Y

tensor([ 1,  3,  6, 10])

Quite simple for now, right ?
To setup our notations, we will keep this example for a bit.

Here, we use an accumulator, `cumulative_sum`, which we update as we go through the input array `X`.

An equivalent way to rewrite the above code is :

In [4]:
Y = torch.zeros_like(X)

Y[0] = X[0]
for t in range(1, X.size(0)):
    Y[t] = Y[t-1] + X[t]

Y

tensor([ 1,  3,  6, 10])

While not explicitly present, the accumulator is still here, in `Y`. It is propagated with the recurrence relation `Y[t] = Y[t-1] + X[t]`.

We can visualize what happens with a simple diagram, which should remind you a bit about RNNs :

<p align="center">
    <img src="assets/cumsum_rnns.jpg" alt="cumulative sum" width="1000" height="300" alt="python mamba"/>
</p>

In some sense, `Y` plays the role of the hidden state, while `X` plays the role of the input : as we process the input, we keep and update a running hidden state.

We see that this method of computation, which uses a sequential loop, induces `L` sequential steps of computations in order to compute the whole output `Y`.

Now, is it possible to <b>parallelize</b> this scan operation ?

This is just what does the <b>parallel scan</b>. 

Let's stay with the simple example of our cumulative sum. In fact, let's simplify it even more : let's say with just want to compute the sum of our input array `X`.
Again, we could come up with a for loop to count the elements. But can't we parallelize this computation ?

Yes, and it is best visualized with this simple tree :

<p align="center">
    <img src="assets/reduction_tree.jpg" alt="cumulative sum" width="1000" height="600" alt="python mamba"/>
</p>

If we assume that the length of our array is a power of 2, then we just have to group the elements 2 by 2, add them, and repeat until we are left with one element, our result. If `L` is `2**d`, then we will need to do `math.log2(L)` sequential steps to compute the sum of the array. That's a major speedup over the `L` steps of the naive for loop.

How could we implement this in Python ?

In [5]:
X = torch.tensor([1, 2, 3, 4, 5, 6, 7, 8]) # input array
L = X.size(0)

We can <i>group the elements by two</i> using :

In [6]:
Xa = X.view(L//2, 2)
Xa

tensor([[1, 2],
        [3, 4],
        [5, 6],
        [7, 8]])

We now have pairs of elements. We can do :

In [7]:
Xa[:, 0]

tensor([1, 3, 5, 7])

and

In [8]:
Xa[:, 1]

tensor([2, 4, 6, 8])

to access the elements from the two groups. We can see that, to compute the first step, we simply need to sum these two arrays :

In [9]:
Xa[:, 0] + Xa[:, 1]

tensor([ 3,  7, 11, 15])

Yay ! We have just accomplished our first step ! Now, we just need to repeat what we've just done.

Note that we could do `Xa = Xa[:, 0] + Xa[:, 1]` and then we would just need to repeat the previous step with `Xa`. But :
- this will allocate extra memory spaces for storing the result of the first step (`Xa` currently shares the data as `X`, doing this would compute the sum and store its output in a new memory space).
- we will reuse some of these values, later, for the full scan operation.

Hence, we will work by updating `X` <b>in-place</b>, by doing :

In [10]:
Xa[:, 1] += Xa[:, 0]
Xa[:, 1]

tensor([ 3,  7, 11, 15])

This is what we want. Now, we will repeat the step we have just done, but on `Xa[:, 1]` rather than on `X`.

In [11]:
Xa = Xa[:, 1]
Xa = Xa.view(Xa.size(0)//2, 2)

Again, `Xa` is split in two groups :

In [12]:
Xa

tensor([[ 3,  7],
        [11, 15]])

We sum these two groups, and put the result in the second half of `Xa`:

In [13]:
Xa[:, 1] += Xa[:, 0]
Xa[:, 1]

tensor([10, 26])

We have two elements left ! This means, only one more step to go (because `math.log2(2) = 1` of course !) :

In [14]:
Xa = Xa[:, 1]
Xa = Xa.view(Xa.size(0)//2, 2)

Xa[:, 1] += Xa[:, 0]
Xa[:, 1]

tensor([36])

Which is the result we want !

To recap, here is the full code :

In [17]:
X = torch.tensor([1, 2, 3, 4, 5, 6, 7, 8]) # input array
L = X.size(0)

Xa = X

for k in range(int(math.log2(L))):
    T = 2 * (Xa.size(0) // 2)

    # split into 2 groups of pairs of elements
    Xa = Xa.view(T//2, 2) 

    # for each pair, add the first to the second
    Xa[:, 1].add_(Xa[:, 0])

    # change the view for the next iteration
    Xa = Xa[:, 1]

X

tensor([ 1,  3,  3, 10,  5, 11,  7, 36])

Our result is found in the last element of `X` (remember that all the `Xa`'s are views of our original tensor!)

Let's look at how `X` evolved during this whole process : each line corresponds to the data of tensor `X` at a particular iteration.

<p align="center">
    <img src="assets/tensor_mem.jpeg" alt="tensor evolution in memory" width="1000" height="500" alt="python mamba"/>
</p>

To help clarify, we can also draw on top of this the tree underlying the operation.

<p align="center">
    <img src="assets/tensor_mem_tree.jpeg" alt="tensor evolution in memory" width="1000" height="500" alt="python mamba"/>
</p>

Note that having all these numbers is a bit cumbersome, as only a few of them changes at each iteration.

So, for the rest of this document, we'll represent this with the following diagram :

<p align="center">
    <img src="assets/tree_reduction.jpeg" alt="tensor evolution in memory" width="1000" height="500" alt="python mamba"/>
</p>

You can see that this is the exact same tree that was drawn in red in the above figure.

It is essentially a simple tree again, **but** :
- the placement of the nodes is important : see how the sum of each pair is placed on the element of the right.
- at each level of the tree, we can easily see the exact state of the array : you just have, for each position, to look at the upmost element in the column.

Ok, so we can compute the sum of an array in parallel. Now, what about the cumulative sum of an array ?

If we look again at the previous tree, we can see that the values of some nodes are partial sums of our input array. For example, the 10 is the sum of elements 1 to 4.

In fact, it's quite easy to see (and demonstrate) that each node holds the sum of the elements of `X` which are its children or more generally its descendants.

We can see it even more clearly here :

<p align="center">
    <img src="assets/tree_reduction_xs.jpeg" alt="tensor evolution in memory" width="1000" height="550" alt="python mamba"/>
</p>

Consedering this, it's now becoming clearer that we will be able to reuse this tree to construct the cumulative sum of `X` : it is filled with partial sums of the input array. That's why we used in-place operations in the code above : after the process of computing the sum, we have left in `X` some values of this tree.

If we look again at this tree, we see that that, for a given node that is <b>a left child</b>, if its parent is a right child, then adding the value of the left child (the "left brother" of its parent, if you will) to the original node will extend its partial sum. It's best visualized here, with adding $x_{1->4}$ to $x_{5->6}$, which will give us $x_{1->6}$.

# schema flow down-sweep

Let's say we do this computation from top to bottom, for all nodes that are <b>left children</b> :

# schema autres computations

This is good, some values are already cumulative sums ($x_{1->3}$ and $x_{1->7}$), some are partials ($x_{3->5}$) and somes are unchanged.

But, and this is were it is a bit tricky, if we account for the fact that this isn't a static tree but a representation of the evolution of our tensor in memory, we know that we execute this sequence of computations (called the <b>down-sweep</b>, because we go from top to bottom) <b>after</b> the up-sweep (where we went from bottom to top). Hence, if we look at the fourth value of our tensor for example, during the down-sweep, it is already equal to $x_{1->4}$ (remember that we only do in-place operations). Same for the last value : it isn't equal to $x_8$, but to $x_{1->8}$ after the <b>up-sweep</b>.

# schema autres computations updated

This is not yet a proof (the proof is left to the reader 😉), but essentially, we can see that the descendants of the two nodes we're summing would combine and we would get the sum of all the descendants up to the <b>left child</b> we are considering. 

For a full and rigourous proof, we would need to actually modify a bit the formulation of our algorithm (but it would stay strictly equivalent) : we consider two trees, one built by the up-sweep operation, and the other built by the down-sweep operation, as follows :

- start by placing as the root the top element of the up sweep tree (ie, the sum)
- then, pass as its right child its value, and as its left child its value minus the value of the corresponding right child in the up-sweep tree
- repeat, until the number of nodes equals the length of the input array/



This is what we get for the example :

# schema up-sweep and down-sweep tree example

This is the rule we followed to construct the down-sweep tree :

# schema rule

We could then show, by induction, that : <i>After a complete down-sweep, each node of the (down-sweep) tree contains the sum of all the leaf values that <b>precede it</b></i> (not necessarily descendants).

proof

This is the same approach followed by Guy E. Blelloch in <i>Prefix Sums and Their Applications</i>. However, as you can see, I chose to followed a more algorithm-friendly approach and keep this math-friendly approach for the proof. Blelloch follows the principle of an exclusive scan, while here, it is an inclusive scan that is presented (the cumulative sum take into account only elements that are strictly before it)

left to write :
- python cum sum
- introduction de A
- code pscan.py forward

- backward ?
- code pscan.py backward